In [1]:
import requests
import pandas as pd

request_endpoint = "https://examen-api.s3.eu-west-1.amazonaws.com/Students"
response = requests.get(request_endpoint)

student_list = response.json()['StudentList']
print ("student list",student_list)

student list [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [2]:
request_endpoint = "https://examen-api.s3.eu-west-1.amazonaws.com/Student/{ID}"
ID = 0
request_student = request_endpoint.format(ID=ID)
response = requests.get(request_student).json()
print(response)

# process students
def extract_enrollments (list_of_students) :
    students = []
    for ID in list_of_students :
        request_student = request_endpoint.format(ID=ID)
        student = requests.get(request_student).json()
        students.append(student)
    # create students dataframe
    df = pd.DataFrame(students)
    # explode StudentCourses
    df = df.explode("StudentCourses")
    df.rename({"StudentCourses":"CourseID"},axis=1,inplace=True)
#   CourseID as int
    df.CourseID = df.CourseID.astype(int)
    # reorder columns
    new_cols = ["CourseID","StudentID","StudentName","StudentCursus"]
    df = df[new_cols]
    
    return df

{'StudentID': 0, 'StudentName': 'John Chambers', 'StudentCursus': 'DS', 'StudentCourses': [3, 2, 4, 1]}


In [3]:
enrollments = extract_enrollments(student_list)
print (enrollments.dtypes)
display (enrollments)

CourseID          int64
StudentID         int64
StudentName      object
StudentCursus    object
dtype: object


,CourseID,StudentID,StudentName,StudentCursus
0,3,0,John Chambers,DS
0,2,0,John Chambers,DS
0,4,0,John Chambers,DS
0,1,0,John Chambers,DS
1,4,1,David Williams,DA
...,...,...,...,...
98,5,98,Kevin Cortinas,DE
99,5,99,Jennifer Reighard,DE
99,2,99,Jennifer Reighard,DE
99,0,99,Jennifer Reighard,DE


In [4]:
# Insérez votre code ici
# process attendances
def extract_attendances (list_of_students) :
    request_endpoint = "https://examen-api.s3.eu-west-1.amazonaws.com/Attendance/{ID}"
    students = []
    for ID in list_of_students :
        request_student = request_endpoint.format(ID=ID)
        student = requests.get(request_student).json()
        students.append(student)
    # create students dataframe
    df = pd.DataFrame(students)
    # explode StudentAttendance
    col = 'StudentAttendance'
    df = df.explode(col).reset_index(drop=True)
    df['Date'] = df[col].apply(lambda event: list(event.values())[0])
    df['CourseID'] = df[col].apply(lambda event: list(event.keys())[0])
    #   CourseID as int
    df["CourseID"] = df.CourseID.astype(int)
#     process Date
    df['Date'] = pd.to_datetime(df['Date'],format = "%d-%m-%Y")
    print (df.dtypes)
    # reorder columns
    new_cols = ["CourseID","StudentID","Date"]
    df = df[new_cols]

    return df

results = extract_attendances(student_list[0:2])
display(results.shape)
display(results.head(10))

StudentID                     int64
StudentAttendance            object
Date                 datetime64[ns]
CourseID                      int64
dtype: object


(58, 3)

,CourseID,StudentID,Date
0,2,0,2022-10-17
1,3,0,2022-09-19
2,2,0,2022-10-10
3,2,0,2022-09-12
4,3,0,2022-11-07
5,1,0,2022-11-07
6,3,0,2022-10-03
7,4,0,2022-11-07
8,2,0,2022-09-26
9,1,0,2022-09-05


In [5]:
attendances = extract_attendances(student_list)
display (attendances)

StudentID                     int64
StudentAttendance            object
Date                 datetime64[ns]
CourseID                      int64
dtype: object


,CourseID,StudentID,Date
0,2,0,2022-10-17
1,3,0,2022-09-19
2,2,0,2022-10-10
3,2,0,2022-09-12
4,3,0,2022-11-07
...,...,...,...
2954,0,99,2022-09-26
2955,0,99,2022-09-19
2956,2,99,2022-11-07
2957,3,99,2022-09-26


In [32]:
# Insérez votre code ici
# process grades
def extract_grades (list_of_students) :
    request_endpoint = "https://examen-api.s3.eu-west-1.amazonaws.com/Grades/{ID}"
    students = []
    for ID in list_of_students :
        request_student = request_endpoint.format(ID=ID)
        student = requests.get(request_student).json()
        students.append(student)
    # create students dataframe
    df = pd.DataFrame(students)
    # explode StudentAttendance
    col = 'StudentGrades'
    df = df.explode(col).reset_index(drop=True)
    df['Grade'] = df[col].apply(lambda event: list(event.values())[0])
    df['CourseID'] = df[col].apply(lambda event: list(event.keys())[0])
    #   CourseID as int
    df["CourseID"] = df.CourseID.astype(int)
    #   Grade as float
    df["Grade"] = df.Grade.astype(float)
    df['Attended'] = df["Grade"].apply(lambda value: True if value > 0.0 else False)
    df['Success'] = df["Grade"].apply(lambda value: True if value >= 10.0 else False)

    # reorder columns
    new_cols = ["CourseID","StudentID","Grade","Attended","Success"]
    df = df[new_cols]

    return df

results = extract_grades(student_list[0:3])
display(results.shape)
display(results)

(12, 5)

,CourseID,StudentID,Grade,Attended,Success
0,3,0,8.6,True,False
1,2,0,15.4,True,True
2,4,0,10.2,True,True
3,1,0,8.2,True,False
4,4,1,0.0,False,False
5,2,1,12.9,True,True
6,1,1,12.1,True,True
7,3,1,8.7,True,False
8,3,2,0.0,False,False
9,4,2,16.0,True,True


In [ ]:
grades = extract_grades(student_list)
print ("results different from solutions, but seems OK for me")
display (grades)

,CourseID,StudentID,Grade,Attended,Success
0,3,0,8.6,True,False
1,2,0,15.4,True,True
2,4,0,10.2,True,True
3,1,0,8.2,True,False
4,4,1,0.0,False,False
...,...,...,...,...,...
395,5,98,10.2,True,True
396,5,99,10.5,True,True
397,2,99,13.0,True,True
398,0,99,8.5,True,False


In [ ]:
def transform_enrollments (df) :

    # recover most frequent cursus per courseID
    MajorityCursus = df.groupby(["CourseID"])["StudentCursus"].agg(lambda x: pd.Series.mode(x)[0])
  
    results = df.groupby(["CourseID"]).agg(EnrolledStudents = ("StudentID","count"))
    results ["MajorityCursus"]=MajorityCursus.values

    return results.reset_index()

results = transform_enrollments(enrollments)
display(results)


[3 2 4 1 5 0]


,CourseID,EnrolledStudents,MajorityCursus
0,0,66,DS
1,1,62,DA
2,2,66,DA
3,3,60,DA
4,4,71,DA
5,5,75,DS


In [33]:
def transform_attendances (df) :

    # group by StudentID and CourseID
  
    tmp= df.groupby(["StudentID","CourseID"]).agg( AttendanceRate =("Date" ,"count"))/10.0
    results = tmp.reset_index().groupby("CourseID")["AttendanceRate"].mean()
    return results.reset_index()

results = transform_attendances(attendances)
display(results)


,CourseID,AttendanceRate
0,0,0.710606
1,1,0.822581
2,2,0.759091
3,3,0.681667
4,4,0.726761
5,5,0.738667


In [ ]:
def transform_grades (df) :


    # compute temporary attributes
    tmp= df.groupby(["CourseID"]).agg( ExamSum =("Grade" ,"sum"),
                                      PostiveGradeCount = ("Grade",lambda x: len(x[x>0])),
                                      GradeSuccessCount = ("Grade",lambda x: len(x[x>=10.0])),
                                      CourseCount = ("Grade" , "count"))
    # compute new attribute
    tmp["ExamAverage"] = tmp.ExamSum/tmp.PostiveGradeCount
    tmp["ExamSuccessRate"] = tmp.GradeSuccessCount/tmp.CourseCount
    tmp["ExamAttendanceRate"] = tmp.PostiveGradeCount/tmp.CourseCount

    tmp_df = tmp.reset_index()

    # reorder columns
    new_cols = ["CourseID","ExamAttendanceRate","ExamSuccessRate","ExamAverage"]
    tmp_df = tmp_df[new_cols]
    # process exam success rate

    return tmp_df

results = transform_grades(grades)
display(results)

length 400


,CourseID,ExamAttendanceRate,ExamSuccessRate,ExamAverage
0,0,0.909091,0.378788,9.278333
1,1,0.919355,0.354839,9.021053
2,2,0.909091,0.787879,13.406667
3,3,0.900000,0.583333,11.212963
4,4,0.873239,0.436620,10.256452
5,5,0.880000,0.520000,10.574242


In [67]:
results = transform_enrollments(enrollments)
results = results.merge(transform_attendances(attendances),on="CourseID")
results = results.merge(transform_grades(grades),on="CourseID")

display(results)

[3 2 4 1 5 0]
length 400


,CourseID,EnrolledStudents,MajorityCursus,AttendanceRate,ExamAttendanceRate,ExamSuccessRate,ExamAverage
0,0,66,DS,0.710606,0.909091,0.378788,9.278333
1,1,62,DA,0.822581,0.919355,0.354839,9.021053
2,2,66,DA,0.759091,0.909091,0.787879,13.406667
3,3,60,DA,0.681667,0.900000,0.583333,11.212963
4,4,71,DA,0.726761,0.873239,0.436620,10.256452
5,5,75,DS,0.738667,0.880000,0.520000,10.574242


In [ ]:
print("it was good to have the results in order to find out the solutions")